In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


In [ ]:

# Set up paths to your dataset
train_dir = 'CNN_dataset/training_set'
val_dir = 'CNN_dataset/test_set'

# Image data generators
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150, 150),
                                                    batch_size=32,
                                                    class_mode='binary')

val_generator = val_datagen.flow_from_directory(val_dir,
                                                target_size=(150, 150),
                                                batch_size=32,
                                                class_mode='binary')

# Building the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compiling the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Training the model
history = model.fit(train_generator,
                    epochs=25,
                    validation_data=val_generator)

# Save the model after training
model.save('CNN_model/cnn_cat_or_dog_v4.0.keras')

# Evaluating the model
loss, accuracy = model.evaluate(val_generator)
print(f'Validation Loss: {loss}')
print(f'Validation Accuracy: {accuracy}')


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the saved model
model = tf.keras.models.load_model('CNN_model/cnn_cat_or_dog_v4.0.keras')

# Load and preprocess the image
img_path = 'CNN_dataset/single_prediction/cat_or_dog_7.jpg'
img = image.load_img(img_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array /= 255.0  # Normalize the image to the same scale as training data

# Make a prediction
prediction = model.predict(img_array)

# Interpret the result
if prediction[0] > 0.5:
    print(f'Dog. \nConfidence: {prediction[0][0]*100:.0f} %')
else:
    print(f'Cat. \nConfidence: {(1 - prediction[0][0])* 100:.0f} %')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Cat. 
Confidence: 94 %
